# Global Developer Brain Drain 2024: Mapping Talent Flow

**Author:** Abderrahmane CHIBOUT, Aida HANAD, Rania MEDLES
**Generated:** 2025-11-29 

----

## Abstract

The global movement of highly-skilled software developers has significant economic consequences for both talent-exporting and -importing countries. Using the Stack Overflow Developer Survey 2024, this notebook builds a reproducible, visualization-first analysis to map "brain drain" patterns. The work includes robust cleaning, feature engineering (skill score, migration intent variables), EDA, and publication-quality visualizations using Tableau.

----

## Notebook structure

1. Data Loading
2. Professional Data Cleaning
3. Feature Engineering
4. Exploratory Data Analysis (EDA)
5. Visualizations
6. Insights & Export


In [1]:
# Setup: imports and display settings

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import pycountry
import os
import json

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

print('Imports loaded')


Imports loaded


## Data Loading

In [2]:
# Data loading: adjust path if needed
survey_results_public_PATH =  './Data/survey_results_public.csv'
survey_results_schema_PATH = './Data/survey_results_schema.csv'
df = pd.read_csv(survey_results_public_PATH)
df.head()


,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,TechDoc,YearsCode,YearsCodePro,DevType,OrgSize,PurchaseInfluence,BuyNewTool,BuildvsBuy,TechEndorse,Country,Currency,CompTotal,LanguageHaveWorkedWith,LanguageWantToWorkWith,LanguageAdmired,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,DatabaseAdmired,PlatformHaveWorkedWith,PlatformWantToWorkWith,PlatformAdmired,WebframeHaveWorkedWith,WebframeWantToWorkWith,WebframeAdmired,EmbeddedHaveWorkedWith,EmbeddedWantToWorkWith,EmbeddedAdmired,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,MiscTechAdmired,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,ToolsTechAdmired,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,NEWCollabToolsAdmired,OpSysPersonal use,OpSysProfessional use,OfficeStackAsyncHaveWorkedWith,OfficeStackAsyncWantToWorkWith,OfficeStackAsyncAdmired,OfficeStackSyncHaveWorkedWith,OfficeStackSyncWantToWorkWith,OfficeStackSyncAdmired,AISearchDevHaveWorkedWith,AISearchDevWantToWorkWith,AISearchDevAdmired,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOHow,SOComm,AISelect,AISent,AIBen,AIAcc,AIComplex,AIToolCurrently Using,AIToolInterested in Using,AIToolNot interested in Using,AINextMuch more integrated,AINextNo change,AINextMore integrated,AINextLess integrated,AINextMuch less integrated,AIThreat,AIEthics,AIChallenges,TBranch,ICorPM,WorkExp,Knowledge_1,Knowledge_2,Knowledge_3,Knowledge_4,Knowledge_5,Knowledge_6,Knowledge_7,Knowledge_8,Knowledge_9,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Frustration,ProfessionalTech,ProfessionalCloud,ProfessionalQuestion,Industry,JobSatPoints_1,JobSatPoints_4,JobSatPoints_5,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,Under 18 years old,"Employed, full-time",Remote,Apples,Hobby,Primary/elementary school,Books / Physical media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have never visited Stack Overflow or the Sta...,NaN,NaN,NaN,NaN,NaN,Yes,Very favorable,Increase productivity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,API document(s) and/or SDK document(s);User gu...,20,17,"Developer, full-stack",NaN,NaN,NaN,NaN,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Dynamodb;MongoDB;PostgreSQL,PostgreSQL,PostgreSQL,Amazon Web Services (AWS);Heroku;Netlify,Amazon Web Services (AWS);Heroku;Netlify,Amazon Web Services (AWS);Heroku;Netlify,Express;Next.js;Node.js;React,Express;Htmx;Node.js;React;Remix,Express;Node.js;React,NaN,NaN,NaN,NaN,NaN,NaN,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,PyCharm;Visual Studio Code;WebStorm,PyCharm;Visual Studio Code;WebStorm,PyCharm;Visual Studio Code;WebStorm,MacOS;Windows,MacOS,NaN,NaN,NaN,Microsoft Teams;Slack,Slack,Slack,NaN,NaN,NaN,Stack Overflow for Teams (private knowledge sh...,Multiple times per day,Yes,Multiple times per day,Quickly finding code solutions;Finding reliabl...,"Yes, definitely","No, and I don't plan to",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Individual contributor,17.0,Agree,Disagree,Agree,Agree,Agree,Neither agree nor disa

In [3]:
# Quick data audit
print('Columns:', len(df.columns))
print(df.columns.tolist())
df.info()

# Missingness overview
missing = df.isna().mean().sort_values(ascending=False)
missing.head(40)


Columns: 114
['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWan

AINextMuch less integrated       0.982456
AINextLess integrated            0.964011
AINextNo change                  0.809007
AINextMuch more integrated       0.794642
EmbeddedAdmired                  0.744288
EmbeddedWantToWorkWith           0.731039
EmbeddedHaveWorkedWith           0.660528
ConvertedCompYearly              0.641869
AIToolNot interested in Using    0.626908
AINextMore integrated            0.626694
Knowledge_9                      0.577685
Frequency_3                      0.576539
Knowledge_8                      0.575806
ProfessionalTech                 0.575714
Knowledge_7                      0.575500
Knowledge_6                      0.574186
Knowledge_5                      0.573941
Knowledge_2                      0.571787
Knowledge_4                      0.571649
Knowledge_3                      0.570656
Frustration                      0.568272
Frequency_2                      0.566545
Frequency_1                      0.566469
ProfessionalCloud                0

## Cleaning

Steps performed:
1. Keep only relevant columns
2. Standardize names
3. Convert numeric fields (salary, experience)
4. Parse multi-select columns into lists
5. Normalize countries and map to ISO3
6. Remove or mark outliers
7. Save cleaned dataset


## Rania
 Please, after each step explain the choises and decisions u made, like why u removed somthing or how u filled missings ect....
### Merciii 

In [ ]:
# 1) Select relevant columns

# Keep columns that exist in the dataset

# 2) Rename to pythonic names

# 3) Salary cleaning: convert all to usd

# Remove extreme values and outliers 

# 4) Convert to numeric if neccery

# 5) Parse multi-select columns into lists

# 6) Standardize country names & map to ISO3

# Manual fixes for common mismatches

# 7) Feature: num_languages and total_tech_count

# 8) Drop rows missing core values

# Rania, u can add any step if u see it necessery, smart girl.

df_clean = [] # 


In [ ]:
# Feature engineering


## Exploratory Data Analysis (EDA)

In [ ]:
# EDA

# Aida , nti hna mmb3d tji daltak ya bo9o9o
# mazal mkhmtch fi paln ll EDA, but in general ra7 tkon much of aggregations

## Visualization

In [ ]:
# Visualization using python (primery step)

## Exporting

In [ ]:
# Export cleaned dataset and figures
OUT_DIR = 'output'
os.makedirs(OUT_DIR, exist_ok=True)
clean_path = os.path.join(OUT_DIR, 'survey_2024_cleaned.csv')
df_clean.to_csv(clean_path, index=False)
print('Cleaned data saved to', clean_path)

# Save choropleth if fig exists
try:
    fig.write_html(os.path.join(OUT_DIR, 'choropleth_high_skill_share.html'))
    print('Saved example figure to output/')
except Exception as e:
    print('No fig to save or error:', e)


## Insights & Reporting


1. Key findings :top exporting countries, top importers, net flow scores
2. Actionable recommendations (for governments, companies, universities)
3. Limitations and caveats (survey bias, self-reported data)


